In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import time

In [2]:
# 벡터화된 환경 만들기 (학습 빠르게!)
env = make_vec_env("LunarLander-v3", n_envs=8, seed=42)  # 8개 병렬 학습

# PPO 모델 생성 (이 하이퍼파라미터가 지금 제일 잘 됨)
model = PPO(
    "MlpPolicy",           # MLP 네트워크
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=1000,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    tensorboard_log=None
)

# 학습 시작 (총 100만 스텝 ≈ 10~15분 정도면 착륙 성공함!)
print("Train Start! Background training...")
model.learn(total_timesteps=1_000_000, log_interval=10)

# 모델 저장
model.save("ppo_lunar_lander_v3")
print("Finished model saved: ppo_lunar_lander_v3.zip")

Using cpu device
학습 시작! 창 안 띄워도 백그라운드로 학습 중...
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 241         |
|    ep_rew_mean          | -56.4       |
| time/                   |             |
|    fps                  | 994         |
|    iterations           | 10          |
|    time_elapsed         | 80          |
|    total_timesteps      | 80000       |
| train/                  |             |
|    approx_kl            | 0.010474881 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | 0.792       |
|    learning_rate        | 0.0003      |
|    loss                 | 23.5        |
|    n_updates            | 90          |
|    policy_gradient_loss | -0.0102     |
|    value_loss           | 90.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |  

In [1]:
# 학습된 모델 로드해서 보기
model = PPO.load("ppo_lunar_lander_v3")

env = gym.make("LunarLander-v3", render_mode="human", enable_wind=False)
obs, _ = env.reset()

for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)  # 최적 행동
    print(f"{action}")
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()

    if terminated or truncated:
        print("에피소드 종료! 리셋...")
        obs, _ = env.reset()
        time.sleep(1)  # 착륙 보고 싶으면 잠깐 멈춤

env.close()

NameError: name 'PPO' is not defined